In [2]:

import pandas as pd
import requests

# Load the crop dataset
df = pd.read_csv(r"C:\Users\Atharv\Documents\Capstone Project\crop_master_organic.csv")

# Function to fetch live weather using OpenWeatherMap
def get_weather(city, api_key):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    data = response.json()
    if response.status_code != 200 or 'main' not in data:
        return None, f" Could not fetch weather for {city}."
    temp = data['main']['temp']
    return temp, None

# Main recommendation function
def recommend_crop_path(state, district, city, crop_name, api_key):
    crop_data = df[df["Crop_Name"].str.lower() == crop_name.lower()]
    if crop_data.empty:
        return f" Crop '{crop_name}' not found in database."

    crop = crop_data.iloc[0]
    temperature, error = get_weather(city, api_key)
    if error:
        return error

    min_temp = crop['Temperature_Min_C']
    max_temp = crop['Temperature_Max_C']
    temp_ok = min_temp <= temperature <= max_temp

    output = []
    output.append(f" Crop: Organic {crop['Crop_Name']}")
    output.append(f" Location: {district}, {state}")
    output.append(f" Current Temperature in {city}: {temperature}°C " +
                  (" Ideal" if temp_ok else f" Not in ideal range ({min_temp}–{max_temp}°C)"))
    if not temp_ok:
        output.append(" Weather is not ideal, but here’s the full guide:")

    output.extend([
        f"\n Sowing Season: {crop['Sowing_Season']}",
        f" Soil Type: {crop['Ideal_Soil']} (pH {crop['Ideal_Soil_pH']})",
        f" Watering: {crop['Watering_Frequency']}",
        f" Fertilizers: {crop['Organic_Fertilizers']}",
        f" Pest Management: {crop['Pest_Management']}",
        f" Harvest Time: {crop['Harvest_Days']} days",
        f" Growth Stages: {crop['Growth_Stages_Detailed']}"
    ])

    return "\n".join(output)

#  Example usage
api_key = "e9d5e3b92ecf1f26b5f2742d202d1de6"
print(recommend_crop_path("Maharashtra", "Nashik", "Nashik", "Tomato", api_key)) 


 Crop: Organic Tomato
 Location: Nashik, Maharashtra
 Current Temperature in Nashik: 20.67°C  Ideal

 Sowing Season: June–July
 Soil Type: Loamy (pH Sandy loam (pH 6.0–6.8))
 Watering: Every 3 days
 Fertilizers: Vermicompost, Neem Cake
 Pest Management: Spray Neem Oil (weekly)
 Harvest Time: 90 days
 Growth Stages: Seedling: 0–15, Vegetative: 16–35, Flowering: 36–55, Harvest: 56–90
